In [1]:
import time
import math
import imutils
import os
import sys
import numpy as np
import cv2
#sys.path.append(os.path.dirname(__file__) + "/../")
from scipy.misc import imread, imsave
from skimage.measure import structural_similarity as ssim
from config import load_config
from dataset.factory import create as create_dataset
from nnet import predict
from util import visualize
import cv2
from dataset.pose_dataset import data_to_input


from multiperson.detections import extract_detections
from multiperson.predict import SpatialModel, eval_graph, get_person_conf_multicut
from multiperson.visualize import PersonDraw, visualize_detections
import matplotlib.pyplot as plt
import tensorflow as tf
from tkinter import messagebox

In [159]:
def vibrate(key):
    os.system('play  --null --channels 1 synth %s sine %f' % (1, 500))
    
    
    
def compare_images(slope1, slope2, allowance):
    for key in slope1:
        print(key,slope1[key]-slope2[key])
        if abs(slope1[key]-slope2[key]) > allowance:
            vibrate(key)
            print("error at : ", key)
            return (key,slope1[key]-slope2[key])
            
def slope_calc(co1, co2):
    body_dict={'right_upper_arm':co1[4],
              'left_upper_arm':co1[5],
               'right_upper_leg':co1[8],
               'left_upper_leg':co1[9]
              }
    slopes={}
    slopes_user={}
    body_dict['backbone_top']=np.array([int((body_dict['right_upper_arm'][0]+body_dict['left_upper_arm'][0])/2),
                           int((body_dict['right_upper_arm'][1]+body_dict['left_upper_arm'][1])/2)])
    body_dict['backbone_bottom']=np.array([int((body_dict['right_upper_leg'][0]+body_dict['left_upper_leg'][0])/2),
                           int((body_dict['right_upper_leg'][1]+body_dict['left_upper_leg'][1])/2)])

    body_dict_lines={
        'backbone':(body_dict['backbone_top'], body_dict['backbone_bottom']),
        'nose_right':(co1[0],co1[1]),
        'nose_left': (co1[0], co1[2]),
        'right_eye_ear': (co1[1], co1[3]),
        'left_eye_ear':(co1[2],co1[4]),
        'right_upper_arm':(co1[5],co1[7]),
        'left_upper_arm':(co1[6],co1[8]),
        'right_forearm': (co1[7],co1[9]),
        'left_forearm': (co1[8],co1[10]),
        'right_upper_leg':(co1[11],co1[13]),
        'left_upper_leg':(co1[12],co1[14]),
        'right_shin':(co1[13],co1[15]),
        'left_shin':(co1[14],co1[16])
    }


    for key in body_dict_lines:
        a=math.atan((body_dict_lines['backbone'][1][1]-body_dict_lines['backbone'][0][1])/(body_dict_lines['backbone'][0][0]-body_dict_lines['backbone'][1][0]))
        slopes[key]=(math.atan((body_dict_lines[key][1][1]-body_dict_lines[key][0][1])/(body_dict_lines[key][0][0]-body_dict_lines[key][1][0])))-a
    
    
    body_dict_user={'right_upper_arm':co2[4],
              'left_upper_arm':co2[5],
               'right_upper_leg':co2[8],
               'left_upper_leg':co2[9]
              }
    body_dict_user['backbone_top']=np.array([int((body_dict_user['right_upper_arm'][0]+body_dict_user['left_upper_arm'][0])/2),
                           int((body_dict_user['right_upper_arm'][1]+body_dict_user['left_upper_arm'][1])/2)])
    body_dict_user['backbone_bottom']=np.array([int((body_dict_user['right_upper_leg'][0]+body_dict_user['left_upper_leg'][0])/2),
                           int((body_dict_user['right_upper_leg'][1]+body_dict_user['left_upper_leg'][1])/2)])

    body_dict_lines_user={
        'backbone':(body_dict_user['backbone_top'], body_dict_user['backbone_bottom']),
        'nose_right':(co2[0],co2[1]),
        'nose_left': (co2[0], co2[2]),
        'right_eye_ear': (co2[1], co2[3]),
        'left_eye_ear':(co2[2],co2[4]),
        'right_upper_arm':(co2[5],co2[7]),
        'left_upper_arm':(co2[6],co2[8]),
        'right_forearm': (co2[7],co2[9]),
        'left_forearm': (co2[8],co2[10]),
        'right_upper_leg':(co2[11],co2[13]),
        'left_upper_leg':(co2[12],co2[14]),
        'right_shin':(co2[13],co2[15]),
        'left_shin':(co2[14],co2[16])
    }


    for key in body_dict_lines_user:
        b=math.atan((body_dict_lines_user['backbone'][1][1]-body_dict_lines_user['backbone'][0][1])/(body_dict_lines_user['backbone'][0][0]-body_dict_lines_user['backbone'][1][0]))
        slopes_user[key]=(math.atan((body_dict_lines_user[key][1][1]-body_dict_lines_user[key][0][1])/(body_dict_lines_user[key][0][0]-body_dict_lines_user[key][1][0])))-a
    
    
    
    
    return slopes, slopes_user

In [160]:
a,b=slope_calc(co1, user_co1)
compare_images(a,b,0.5)

backbone 0.39833239483010896
nose_right 0.03123983343026837
nose_left 0.2110933332227467
right_eye_ear -0.08314123188844125
left_eye_ear 0.16037544397595072
right_upper_arm 0.4248321629193428
left_upper_arm 1.3664596475079354
error at :  left_upper_arm


('left_upper_arm', 1.3664596475079354)

In [153]:
slope_calc(user_co1, co1)[0]

{'backbone': 0.0,
 'left_eye_ear': -0.7202164805356259,
 'left_forearm': -2.4048048144742307,
 'left_shin': 0.43780767561273026,
 'left_upper_arm': -1.3870399318299746,
 'left_upper_leg': 0.44533104463769546,
 'nose_left': -2.1861358686002887,
 'nose_right': -0.43548604201291374,
 'right_eye_ear': -1.4007377052028402,
 'right_forearm': 0.1859303045838463,
 'right_shin': -2.6212198197623344,
 'right_upper_arm': -2.431114531727153,
 'right_upper_leg': -2.6010416262911633}

In [152]:
a

{'backbone': 0.0,
 'left_eye_ear': -0.9581734313897842,
 'left_forearm': -2.840619645996001,
 'left_shin': -3.0163804989027456,
 'left_upper_arm': -0.41891267915214825,
 'left_upper_leg': -2.967644861165743,
 'nose_left': -2.373374930207651,
 'nose_right': -0.8025786034127543,
 'right_eye_ear': -1.8822113319213905,
 'right_forearm': 0.040893719547647756,
 'right_shin': -3.0168760390009357,
 'right_upper_arm': -2.404614763637919,
 'right_upper_leg': 0.027996756235124654}

In [79]:
def run_predict(frame):

    # Load and setup CNN part detector
    #tf.reset_default_graph()
    image= frame
    image_batch = data_to_input(frame)
    
    # Compute prediction_n with the CNN
    outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
    scmap, locref, pairwise_diff = predict.extract_cnn_output(outputs_np, cfg, dataset.pairwise_stats)
    detections = extract_detections(cfg, scmap, locref, pairwise_diff)
    unLab, pos_array, unary_array, pwidx_array, pw_array = eval_graph(sm, detections)
    m=time.time()
    person_conf_multi = get_person_conf_multicut(sm, unLab, unary_array, pos_array)
    img = np.copy(image)
    #coor = PersonDraw.draw()
    visim_multi = img.copy()
    co1=draw_multi.draw(visim_multi, dataset, person_conf_multi, image)
    return pos_array

## start_time=time.time()
cfg = load_config("demo/pose_cfg_multi.yaml")
dataset=create_dataset(cfg)
sm = SpatialModel(cfg)
sm.load()
tf.reset_default_graph()
draw_multi = PersonDraw()
sess, inputs, outputs = predict.setup_pose_prediction(cfg)
fps_time=0
# Read image from file
slopes={}
k=0
cap=cv2.VideoCapture('msgifs/icon4.gif')
cap_user=cv2.VideoCapture('user.mp4')
i=0
while (True):
    ret, orig_frame= cap.read()
    ret2, orig_frame_user= cap_user.read()
    if i%25 == 0:                   
        
        frame = cv2.resize(orig_frame, (0, 0), fx=0.50, fy=0.50)
        #frame=orig_frame
        user_frame=cv2.resize(orig_frame_user, (0, 0), fx=0.50, fy=0.50)
        co1=run_predict(frame)
        user_co1=run_predict(frame)
        try:
            slope_reqd, slope_user=slope_calc(co1, user_co1)
            compare_images(slope_reqd, slope_user, 0.75)
        except IndexError:
            #if len(co1)!=len(user_co1):
            pass
            
        cv2.putText(user_frame,
                    "FPS: %f" % (1.0 / (time.time() - fps_time)),
                    (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 255, 0), 2)
        cv2.imshow('user_frame', user_frame)
        cv2.imshow('frame', frame)
        fps_time=time.time()
        #visualize.waitforbuttonpress()
        if cv2.waitKey(10)==ord('q'):
            break
elapsed= time.time()-start_time
cap.release()
cap_user.release()
cv2.destroyAllWindows()

In [9]:
cap.release()
cap_user.release()
cv2.destroyAllWindows()

TEST ON IMAGES

In [16]:
start_time=time.time()
cfg = load_config("demo/pose_cfg_multi.yaml")
dataset=create_dataset(cfg)
sm = SpatialModel(cfg)
sm.load()
tf.reset_default_graph()
draw_multi = PersonDraw()
sess, inputs, outputs = predict.setup_pose_prediction(cfg)
fps_time=0
# Read image from file
slopes={}
k=0
cap=cv2.VideoCapture('exer.mp4')
cap_user=cv2.VideoCapture(0)
i=0
"""
ret, orig_frame=cap.read()
ret2, orig_frame_user=cap.read()
"""
orig_frame = cv2.imread('sample.jpg')
orig_frame_user = cv2.imread('raised.jpg')
frame = cv2.resize(orig_frame, (0, 0), fx=0.50, fy=0.50)
user_frame=cv2.resize(orig_frame_user, (0, 0), fx=0.50, fy=0.50)
co1=run_predict(frame).astype(int)
user_co1=run_predict(user_frame).astype(int)


try:
    slope_reqd=slope_calc(co1)
    slope_user=slope_calc(user_co1)
    compare_images(slope_reqd, slope_user,1)
except IndexError:
    #if len(co1)!=len(user_co1):
    print("Index Error")
    pass

cv2.putText(user_frame,
            "FPS: %f, error : " % (1.0 / (time.time() - fps_time)),
            (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
            (0, 255, 0), 2)

cv2.imshow('user_frame', user_frame)
cv2.imshow('frame', frame)
fps_time=time.time()
#visualize.waitforbuttonpress()
if cv2.waitKey(0)==ord('q'):
    cap.release()
    cap_user.release()
    cv2.destroyAllWindows()
elapsed= time.time()-start_time
#print("sse score : ", sse)


INFO:tensorflow:Restoring parameters from models/coco/coco-resnet-101


INFO:tensorflow:Restoring parameters from models/coco/coco-resnet-101


num_people:  1
#tracked objects: 0
num_people:  1
#tracked objects: 1
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [12]:
cap.release()
cap_user.release()
cv2.destroyAllWindows()

In [118]:
frame=cv2.imread('sample.jpg')
co=run_predict(frame)

num_people:  1
#tracked objects: 1


In [18]:
slopes

{'backbone': 0.0,
 'left_eye_ear': -0.7202164805356259,
 'left_forearm': -2.4048048144742307,
 'left_shin': 0.43780767561273026,
 'left_upper_arm': -1.3870399318299746,
 'left_upper_leg': 0.44533104463769546,
 'nose_left': -2.1861358686002887,
 'nose_right': -0.43548604201291374,
 'right_eye_ear': -1.4007377052028402,
 'right_forearm': 0.1859303045838463,
 'right_shin': -2.6212198197623344,
 'right_upper_arm': -2.431114531727153,
 'right_upper_leg': -2.6010416262911633}

In [102]:
body_dict_lines={
    'backbone':(body_dict['backbone_top'], body_dict['backbone_bottom']),
    'nose_right':(co1[0],co1[1]),
    'nose_left': (co1[0], co1[2]),
    'right_eye_ear': (co1[1], co1[3]),
    'left_eye_ear':(co1[2],co1[4]),
    'right_upper_arm':(co1[5],co1[7]),
    'left_upper_arm':(co1[6],co1[8]),
    'right_forearm': (co1[7],co1[9]),
    'left_forearm': (co1[8],co1[10]),
    'right_upper_leg':(co1[11],co1[13]),
    'left_upper_leg':(co1[12],co1[14]),
    'right_shin':(co1[13],co1[15]),
    'left_shin':(co1[14],co1[16])
}

In [104]:
len(co1)

17

In [105]:
tuple(body_dict_lines['backbone'][0])

(135, 42)